In [5]:
import os
import pandas as pd
import parse
from transliterate import translit

labels = "/home/peterr/macocu/Varieties/BCMS/Denis_hrsrTweets.txt"
additional_labels =  "/home/peterr/macocu/Varieties/BCMS/Denis_hrsrTweets_additional.txt"


In [6]:
p = parse.compile("users/{username}{whitespace:s}{language}\n")
userdict = dict()
with open(labels, "r") as labelfile:
    for line in labelfile.readlines():
        line = translit(line, "sr", reversed=True)
        found = p.parse(line)
        userdict[found["username"]] = found["language"]
with open(additional_labels, "r") as labelfile:
    for line in labelfile.readlines():
        #line = translit(line, "sr", reversed=True)
        found = p.parse(line)
        userdict[found["username"]] = found["language"]

twitter_data_dir = "/home/peterr/macocu/Varieties/BCMS/users"
datafiles = os.listdir(twitter_data_dir)

missing = [filename for filename in datafiles if filename not in userdict.keys()]
len(missing)

386

In [7]:
from bs4 import BeautifulSoup
from typing import List
def read_user_tweets(filename: str) -> List[str]:
    with open(filename, "r") as f:
        content = "<data>"+f.read()+"<\data>"
    data = BeautifulSoup(content, "xml")
    tweets = data.findAll("tweet")
    return [tweet.find("text").text for tweet in tweets]
import re
def remove_mentions(tweet: str) -> str:
    re_pattern = r"(^|[^@\w])@(\w{1,15})\b"
    return re.sub(re_pattern, " ", tweet)
def remove_URL(tweet: str) -> str:
    re_pattern = r"(https?:\/\/|\b)(www\.)?[-a-zA-Z0-9@:%._\+~#=]{1,256}\.[a-z]{2,4}\b([-a-zA-Z0-9@:%_\+.~#?&//=]*)"
    return re.sub(re_pattern, " ", tweet)
def remove_hashtags(tweet: str) -> str:
    re_pattern = r"#\w*[a-zA-Z]\w*"
    return re.sub(re_pattern, " ", tweet)
def remove_retweet_tag(tweet: str) -> str:
    re_pattern = r" RT @(\w{1,15}): "
    return re.sub(re_pattern, " ", tweet)
def to_latin(tweet: str) -> str:
    for lang in ["sr", "mk"]:
        tweet = translit(tweet, lang, reversed=True)
    return tweet

In [8]:
dataset = dict()
for user, language in userdict.items():
    filename = os.path.join(twitter_data_dir, user)
    tweets = read_user_tweets(filename)
    preprocess = lambda s: remove_hashtags(remove_URL(remove_mentions(remove_retweet_tag(to_latin(s)))))
    corrected_tweets = list(map(preprocess, tweets))
    dataset[user] = {"language": language, "tweets": corrected_tweets }

In [9]:
dataset["LeoVinkovic"]


{'language': 'hr',
 'tweets': ['RT : Hahahaha!  i ja iz pera  Sjajno!  ',
  'Kruha & Igara, plakat koji sam dizajnirao za portal   ',
  "'Kruha & Igara' «     via ",
  '"Habitat Pet Kindergarden Visual Identity", a project on  ::   ',
  'Novi vizualni identitet je u portfoliju. "Habitat Pet Kindergarden Visual Identity", a project on  ::   ',
  '"Inchoo Visual Identity Redesign", a project on  ::   ',
  '  hvala ti Luka :)',
  '  thx Boris :)',
  'Portal Dizajn svaki dan napravio je intervju samnom. Hvala Mihaelu i ekipi na tome.  ',
  'Leo Vinković: dobar dizajn je inovativan, zanimljiv i funkcionalan | Dizajn svaki dan:  ',
  'RT : Awesome package design for Honey products by    ',
  '  hvala Tomislave :)',
  'Nakon Notcota, TheDielinea i mnogih drugih, Radosevic med objavljen je i na Student Show-u \n ',
  'Check out my entry to \'s competition "Wacom Talent Audition":  ',
  'RT : Danas jedan osječki   za   i ',
  'Check out new work on my  portfolio: "Špela-Sub"  ',
  '  publish my